In [ ]:
import faiss
import json
import numpy as np
import os

def load_project(path="model2"):
    index = faiss.read_index(os.path.join(path, "index_faiss.bin"))
    
    with open(os.path.join(path, "documents.json"), "r", encoding="utf-8") as f:
        docs = json.load(f)
    
    with open(os.path.join(path, "meta.json"), "r", encoding="utf-8") as f:
        meta = json.load(f)
    
    model_name = meta.get("model", None)

    print(f"✅ Project loaded from folder: {path}")
    return index, docs, model_name


In [152]:
# Function to load the secrets from the local file
def load_secrets(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
secrets = load_secrets('secrets.json')

chatMistralAIKey = secrets['CHAT_MISTRAL_AI_KEY']

In [136]:
index, docs, model_name = load_project()

✅ Project loaded from folder: model2


In [137]:
docs[0]

{'titre': "Explorez les rues de Collonges-la-Rouge accompagnés d'un guide !",
 'description': "Laissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.",
 'longue description': "Laissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.",
 'adresse': 'Office de Tourisme Vallée de la Dordogne, Collonges la Rouge, 1274 Route de Brive, 19500 Collonges-la-Rouge',
 'telephone': '05 65 33 22 00',
 'site Web': 'https://www.vallee-dordogne.com/',
 'dates': [{'begin': '2024-09-22 10:30:00+02:00',
   'end': '2024-09-22 12:00:00+02:00'},
  {'begin': '2024-09-22 15:00:00+02:00', 'end': '2024-09-22 16:30:00+02:00'}],
 'dates affichage': ['dimanche 22 septembre 2024, 10:30 – 12:00',
  'dimanche 22 septembre 2024, 15:00 – 16:30'],
 'prix': "Gratuit. Sans réservation. Départ de l'office de Tourisme à 10h30 et 15h le dima

In [138]:
def get_embedding_text(doc):
    return "\n".join([
        doc.get("titre", ""),
        doc.get("description", ""),
        doc.get("longue description", ""),
        "Adresse : " + doc.get("adresse", ""),
        "Dates : " + ", ".join(doc.get("dates affichage", [])),
        "Prix : " + doc.get("prix", ""),
    ])


In [139]:
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Your embedding model name
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

# Convert your dicts into LangChain Documents
# We'll use the text you want to embed as `page_content` and everything else as metadata
def convert_to_documents(dicts):
    documents = []
    for d in dicts:
        content = get_embedding_text(d)  # Whatever text you used for embeddings
        documents.append(Document(page_content=content, metadata=d))
    return documents

langchain_docs = convert_to_documents(docs[0:500])


vectorstore = FAISS.from_documents(langchain_docs, embedding_model)
retriever = vectorstore.as_retriever()


In [ ]:
import os
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name=model_name)
chunk_size = 500

# Convert your dicts into LangChain Documents
def convert_to_documents(dicts):
    documents = []
    for d in dicts:
        content = get_embedding_text(d)
        documents.append(Document(page_content=content, metadata=d))
    return documents

# Process in chunks
for i in range(0, len(docs), chunk_size):
    chunk = docs[i:i+chunk_size]
    langchain_docs = convert_to_documents(chunk)

    # Build FAISS vectorstore for the chunk
    vectorstore = FAISS.from_documents(langchain_docs, embedding_model)

    # Save FAISS index and metadata
    faiss_folder = f"faiss_index3/faiss_index_chunk_{i//chunk_size}"
    vectorstore.save_local(faiss_folder)
    print(f"Saved {faiss_folder}")


Saved faiss_index2/faiss_index_chunk_0
Saved faiss_index2/faiss_index_chunk_1
Saved faiss_index2/faiss_index_chunk_2
Saved faiss_index2/faiss_index_chunk_3
Saved faiss_index2/faiss_index_chunk_4
Saved faiss_index2/faiss_index_chunk_5
Saved faiss_index2/faiss_index_chunk_6
Saved faiss_index2/faiss_index_chunk_7
Saved faiss_index2/faiss_index_chunk_8
Saved faiss_index2/faiss_index_chunk_9
Saved faiss_index2/faiss_index_chunk_10
Saved faiss_index2/faiss_index_chunk_11
Saved faiss_index2/faiss_index_chunk_12
Saved faiss_index2/faiss_index_chunk_13


In [141]:
vectorstore = FAISS.load_local(
    "faiss_index/faiss_index_chunk_0",
    embedding_model,
    allow_dangerous_deserialization=True
)

for i in range(1, 14):  # Adjust number of chunks
    vs = FAISS.load_local(
        f"faiss_index/faiss_index_chunk_{i}",
        embedding_model,
        allow_dangerous_deserialization=True
    )
    vectorstore.merge_from(vs)
    print(f"Merged chunk {i}")

# Optional: Save final merged vectorstore
vectorstore.save_local("final_faiss_index")


Merged chunk 1
Merged chunk 2
Merged chunk 3
Merged chunk 4
Merged chunk 5
Merged chunk 6
Merged chunk 7
Merged chunk 8
Merged chunk 9
Merged chunk 10
Merged chunk 11
Merged chunk 12
Merged chunk 13


In [153]:
from langchain.prompts import PromptTemplate

# Just "context", because the `stuff` chain will inject the documents
prompt = PromptTemplate(
    input_variables=["context"],
    template=""" {context}"""
)


from langchain.chains import RetrievalQA
from langchain_mistralai.chat_models import ChatMistralAI

llm = ChatMistralAI(api_key=chatMistralAIKey, model="mistral-medium")  # or mistral-medium

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # This just concatenates documents
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

print('ok')

ok


In [143]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BC455FDD20>, search_kwargs={})

In [144]:
# Test a query
query = "Bègles"
response = qa_chain.invoke(query)
print(response["result"])

Voici un résumé structuré des événements mentionnés, avec les informations essentielles pour chaque activité :

---

### **1. Repair' Café (matériel électro)**
**Description** : Atelier collaboratif pour apprendre à réparer des appareils électriques avec des électriciens bénévoles.
**Adresse** : *Le Pied Allez Triez*, 20 avenue Marc Mercier, 24290 Montignac-Lascaux.
**Date** : Samedi 19 octobre 2024, 14h00 – 17h00.
**Prix** : Réservation obligatoire + adhésion à prix libre + participation consciente.

---

### **2. Biblis en folie 2024**
**Description** : Journée culturelle autour du patrimoine local avec :
- Expositions d'objets anciens et insolites.
- Démonstrations de filage de laine.
- Peintures de monuments et sites.
- Balade contée sur les petits commerces historiques.
**Adresse** : Bibliothèque municipale *Gabrielle et Jean Batista*, 25 route de l'Estuaire, 17120 Épargnes.
**Date** : Samedi 28 septembre 2024, 10h00 – 18h00.
**Prix** : Entrée libre.

---

### **3. Jeudi des enfan

In [145]:
# Test a query
query = "Un évènement à Bordeaux autour du vin"
response = qa_chain.invoke(query)
print(response["result"])

Voici un résumé structuré des événements viticoles à Bordeaux et ses environs, avec leurs dates, lieux et informations clés :

---

### **1. 26ème Salon des Vins des Vignerons Indépendants – Bordeaux**
- **Dates** : 14–16 mars 2025
- **Lieu** : Parc des Expositions de Bordeaux (Cours Jules Ladoumègue)
- **Horaires** :
  - Vendredi 14 mars : 10h–20h
  - Samedi 15 mars : 10h–19h
  - Dimanche 16 mars : 10h–18h
- **Détails** :
  - Salon majeur avec 30 000 visiteurs annuels.
  - Dégustations, achats directs auprès des producteurs, concours de vins (médaille d’or, argent, bronze, "coup de cœur").
- **Prix** : Billets en ligne [ici](https://www.vigneron-independant.com/26ème-salon-des-vins-des-vignerons-indépendants-bordeaux).

---

### **2. Portes Ouvertes de Pessac-Léognan**
- **Dates** : 7–8 décembre 2024
- **Lieu** : Pessac-Léognan (divers châteaux)
- **Horaires** : 10h–18h (samedi et dimanche)
- **Détails** :
  - Découverte des châteaux et de leurs vins.
  - Public varié (débutants et am

In [146]:
# Test a query
query = "Un évènement à Bordeaux autour du vin dont la date est en novembre"
response = qa_chain.invoke(query)
print(response["result"])

Voici un résumé structuré des événements que vous avez mentionnés, avec leurs dates, lieux et détails clés :

---

### **1. 26ème Salon des Vins des Vignerons Indépendants – Bordeaux**
- **Dates** : 14–16 mars 2025
- **Lieu** : Parc des Expositions de Bordeaux (Cours Jules Ladoumègue, 33)
- **Horaires** :
  - Vendredi 14 mars : 10h–20h
  - Samedi 15 mars : 10h–19h
  - Dimanche 16 mars : 10h–18h
- **Description** : Salon incontournable pour déguster et acheter des vins directement auprès des producteurs, avec un concours récompensant les meilleurs vins (médaille d'or, argent, bronze, "coup de cœur").
- **Prix** : Non précisé (billetterie en ligne).
- **Site** : [vigneron-independant.com](https://www.vigneron-independant.com/26ème-salon-des-vins-des-vignerons-indépendants-bordeaux)

---

### **2. Portes Ouvertes de Pessac-Léognan**
- **Dates** : 7–8 décembre 2024
- **Lieu** : Châteaux de l’appellation Pessac-Léognan (Pessac, 33)
- **Horaires** : 10h–18h (samedi et dimanche)
- **Descripti

In [147]:
# Test a query
query = ""
response = qa_chain.invoke(query)
print(response["result"])

Voici une synthèse organisée des informations que vous avez fournies sur les différentes activités culturelles et visites proposées :

---

### **1. Visite libre du Musée de la Faïence et des Arts de la table**
- **Description** : Visite libre des collections permanentes et de l'exposition temporaire.
- **Adresse** : 2378 Route d’Hagetmau, 40320 Samadet, France.
- **Date et horaire** : Samedi 17 mai 2025, de 19h00 à 22h30.
- **Prix** : Non précisé.

---

### **2. Atelier : « À la découverte des jeux antiques »**
- **Description** : Découverte en autonomie des jeux pratiqués à l'époque antique.
- **Adresse** : Amphithéâtre gallo-romain, 20 rue Lacurie, 17100 Saintes.
- **Dates et horaires** :
  - Samedi 21 septembre 2024 : 10h00–12h30 et 13h30–18h00.
  - Dimanche 22 septembre 2024 : 10h00–12h30 et 13h30–18h00.
- **Prix** : Gratuit, sans réservation.
- **Contact** : 05 46 97 73 85.

---

### **3. Visite de Nexteam Machining**
- **Description** : Découverte d'une entreprise spécialisée en

In [148]:
response

{'query': '',
 'result': "Voici une synthèse organisée des informations que vous avez fournies sur les différentes activités culturelles et visites proposées :\n\n---\n\n### **1. Visite libre du Musée de la Faïence et des Arts de la table**\n- **Description** : Visite libre des collections permanentes et de l'exposition temporaire.\n- **Adresse** : 2378 Route d’Hagetmau, 40320 Samadet, France.\n- **Date et horaire** : Samedi 17 mai 2025, de 19h00 à 22h30.\n- **Prix** : Non précisé.\n\n---\n\n### **2. Atelier : « À la découverte des jeux antiques »**\n- **Description** : Découverte en autonomie des jeux pratiqués à l'époque antique.\n- **Adresse** : Amphithéâtre gallo-romain, 20 rue Lacurie, 17100 Saintes.\n- **Dates et horaires** :\n  - Samedi 21 septembre 2024 : 10h00–12h30 et 13h30–18h00.\n  - Dimanche 22 septembre 2024 : 10h00–12h30 et 13h30–18h00.\n- **Prix** : Gratuit, sans réservation.\n- **Contact** : 05 46 97 73 85.\n\n---\n\n### **3. Visite de Nexteam Machining**\n- **Descript

In [149]:
docs[0]

{'titre': "Explorez les rues de Collonges-la-Rouge accompagnés d'un guide !",
 'description': "Laissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.",
 'longue description': "Laissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.",
 'adresse': 'Office de Tourisme Vallée de la Dordogne, Collonges la Rouge, 1274 Route de Brive, 19500 Collonges-la-Rouge',
 'telephone': '05 65 33 22 00',
 'site Web': 'https://www.vallee-dordogne.com/',
 'dates': [{'begin': '2024-09-22 10:30:00+02:00',
   'end': '2024-09-22 12:00:00+02:00'},
  {'begin': '2024-09-22 15:00:00+02:00', 'end': '2024-09-22 16:30:00+02:00'}],
 'dates affichage': ['dimanche 22 septembre 2024, 10:30 – 12:00',
  'dimanche 22 septembre 2024, 15:00 – 16:30'],
 'prix': "Gratuit. Sans réservation. Départ de l'office de Tourisme à 10h30 et 15h le dima